In [ ]:
%cd /content
!git clone -b v3.0.1 https://github.com/invoke-ai/InvokeAI # Aug.09, 最新版本为v3.0.1post3
!pip install -q dependency_injector diffusers einops eventlet facexlib flask_cors flask_socketio flaskwebgui getpass_asterisk huggingface-hub
!pip install -q kornia omegaconf pudb pyreadline3 pytorch-lightning realesrgan streamlit taming-transformers-rom1504 test-tube torch-fidelity
!pip install -q torchmetrics transformers picklescan
!pip install -q pillow xformers==0.0.20 triton==2.0.0 -U
!pip install -q git+https://github.com/invoke-ai/GFPGAN@basicsr-1.4.2 #egg=gfpgan
!pip install -q git+https://github.com/openai/CLIP.git@main #egg=clip
!pip install -q git+https://github.com/Birch-san/k-diffusion.git@mps #egg=k-diffusion
!pip install -q git+https://github.com/invoke-ai/clipseg.git@relaxed-python-requirement #egg=clipseg
!pip install -q git+https://github.com/invoke-ai/PyPatchMatch@0.1.4 #egg=pypatchmatch
%mkdir -p /content/InvokeAI/
%cd /content/InvokeAI/
!pip install -q -e .
!wget https://raw.githubusercontent.com/starvskii/InvokeAI-colab-SS/main/INITIAL_MODELS.yaml -O /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml
#Aug.09, default installs models: SD1.5 + LowRA + Canny/Deepth/Openpose + EasyNegative

from google.colab import drive
drive.mount('/content/drive')#Mount google drive/MyDrive
%mkdir -p /content/db/#creat a dir 「db」as target path for the project
%cp -r /content/drive/MyDrive/SD_models/autoimport/ /content/db/#Copy /MyDrive/SD_models/autoimport/ to /content/db/

exit()

In [ ]:
%cd /content/InvokeAI/
!python /content/InvokeAI/scripts/invokeai-model-install.py --root_dir /content/db --yes

import os
import shlex
import subprocess
from pathlib import Path
from typing import Union

id_rsa_file = "/content/InvokeAI/id_rsa"
id_rsa_pub_file = "/content/InvokeAI/id_rsa.pub"
if os.path.exists(id_rsa_file):
    os.remove(id_rsa_file)
if os.path.exists(id_rsa_pub_file):
    os.remove(id_rsa_pub_file)

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

ssh_name = "id_rsa"
ssh_path = Path(os.path.dirname(os.getcwd())) / ssh_name
gen_key(ssh_path)

import threading
def tunnel():
  !ssh -R 80:127.0.0.1:9090 -o StrictHostKeyChecking=no -i /content/id_rsa remote.moe
threading.Thread(target=tunnel, daemon=True).start()

%cd /content/InvokeAI/
!python /content/InvokeAI/scripts/invokeai-web.py --root /content/db